### Simple QSAR example from a 2017 drug design workshop
Danny Barbaro


In [3]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
# from rdkit.Chem.Draw import IPythonConsole
# from rdkit.Chem import Draw
# IPythonConsole.ipython_useSVG=True
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score, matthews_corrcoef
from sklearn.externals import joblib

In [6]:
# logBB is a database of molecules
file_name = "data/logBB.sdf"
molecules = []
y = []
# Get all the molecules out of the database and parse them with the Chem library
for molecule in Chem.SDMolSupplier(file_name):
    if molecule is not None:
        molecules.append(molecule)
        y.append(molecule.GetIntProp("logBB_class"))
fingerprints = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in molecules]

def rdkit_numpy_convert(fingerprints):
    output = []
    for fingerprint in fingerprints:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fingerprint, arr)
        output.append(arr)
    return np.asarray(output)

x = rdkit_numpy_convert(fingerprints)

In [10]:
x.shape
print("Balance:", str(sum(y) / len(y)))

(321, 2048)

# SET SEED

In [12]:
seed = 73

In [38]:
threshold = 0.8

#### Split the whole set on training and test sets

In [13]:
x_training, x_testing, y_training, y_testing = train_test_split(x, y, test_size=0.20, random_state=seed)
cross_validations = StratifiedKFold(n_splits=5, random_state=seed)
for i, (train_index, test_index) in enumerate(cross_validations.split(x_training, y_training)):
    print("\nFold_" + str(i))
    print("TRAIN:", train_index)
    print("TEST:", test_index)
scaler = StandardScaler().fit(x_training)
x_training = scaler.transform(x_training)

## Building a random forest model

In [20]:
# create grid search dictionary
parameter_grid = {"max_features": [x_training.shape[1] // 10, x_training.shape[1] // 7, x_training.shape[1] // 5, x_training.shape[1] // 3], 
              "n_estimators": [100, 200, 300, 400, 500]}
model = GridSearchCV(RandomForestClassifier(), parameter_grid, n_jobs=2, cv=cross_validations, verbose=1)

In [22]:
# run model
model.fit(x_training, y_training)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   26.2s
[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed:   37.9s finished
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=73, shuffle=False),
             error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
          

In [23]:
model.best_params_

{'max_features': 682, 'n_estimators': 100}

In [24]:
model.best_score_

0.80078125

In [25]:
model.cv_results_

{'mean_fit_time': array([0.43746314, 0.85785494, 1.53192692, 0.34266086, 0.83844147,
        1.67527976, 0.38311658, 0.95426869, 1.87873321, 0.4984786 ,
        1.15881963, 2.8201026 ]),
 'std_fit_time': array([0.18937149, 0.20664881, 0.03771478, 0.00383038, 0.0073081 ,
        0.00969643, 0.00795306, 0.01067708, 0.01977272, 0.04383401,
        0.0137222 , 0.4192366 ]),
 'mean_score_time': array([0.01271443, 0.02134485, 0.04073129, 0.01018934, 0.02102442,
        0.04052019, 0.00827141, 0.02067394, 0.05436492, 0.009868  ,
        0.01995416, 0.04068213]),
 'std_score_time': array([0.00455198, 0.00062809, 0.00216496, 0.00075373, 0.00163178,
        0.00157054, 0.00114138, 0.00080743, 0.02938982, 0.00164787,
        0.00097284, 0.00392852]),
 'param_max_features': masked_array(data=[204, 204, 204, 292, 292, 292, 409, 409, 409, 682, 682,
                    682],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
  

In [26]:
model.cv_results_['mean_test_score']

array([0.7734375 , 0.7890625 , 0.77734375, 0.796875  , 0.79296875,
       0.78125   , 0.796875  , 0.7734375 , 0.7890625 , 0.80078125,
       0.78515625, 0.7890625 ])

In [27]:
model.cv_results_['params']

[{'max_features': 204, 'n_estimators': 100},
 {'max_features': 204, 'n_estimators': 250},
 {'max_features': 204, 'n_estimators': 500},
 {'max_features': 292, 'n_estimators': 100},
 {'max_features': 292, 'n_estimators': 250},
 {'max_features': 292, 'n_estimators': 500},
 {'max_features': 409, 'n_estimators': 100},
 {'max_features': 409, 'n_estimators': 250},
 {'max_features': 409, 'n_estimators': 500},
 {'max_features': 682, 'n_estimators': 100},
 {'max_features': 682, 'n_estimators': 250},
 {'max_features': 682, 'n_estimators': 500}]

In [30]:
# apply the scaler to the test set
x_testing = scaler.transform(x_testing)
prediction_random_forest = model.predict(x_testing)

In [32]:
prediction_random_forest

array([1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1])

In [61]:
print("Accuracy = ", accuracy_score(y_testing, prediction_random_forest))
print("MCC = ", matthews_corrcoef(y_testing, prediction_random_forest))
print("Kappa = ", cohen_kappa_score(y_testing, prediction_random_forest))

Accuracy =  0.7230769230769231
MCC =  0.45743323311233
Kappa =  0.44967074317968014


#### Applicability Domain Estimates

In [36]:
prediction_probability = model.predict_proba(x_testing)

In [37]:
prediction_probability

array([[0.03, 0.97],
       [0.14, 0.86],
       [0.61, 0.39],
       [0.4 , 0.6 ],
       [0.54, 0.46],
       [0.05, 0.95],
       [0.01, 0.99],
       [0.29, 0.71],
       [0.57, 0.43],
       [0.12, 0.88],
       [0.35, 0.65],
       [0.27, 0.73],
       [0.08, 0.92],
       [0.27, 0.73],
       [0.31, 0.69],
       [0.96, 0.04],
       [0.37, 0.63],
       [0.56, 0.44],
       [0.33, 0.67],
       [0.93, 0.07],
       [0.  , 1.  ],
       [0.28, 0.72],
       [0.48, 0.52],
       [0.46, 0.54],
       [0.34, 0.66],
       [0.41, 0.59],
       [0.01, 0.99],
       [0.  , 1.  ],
       [0.96, 0.04],
       [0.79, 0.21],
       [0.84, 0.16],
       [0.3 , 0.7 ],
       [0.64, 0.36],
       [0.65, 0.35],
       [0.62, 0.38],
       [0.19, 0.81],
       [0.57, 0.43],
       [0.64, 0.36],
       [0.16, 0.84],
       [0.65, 0.35],
       [0.62, 0.38],
       [0.98, 0.02],
       [0.12, 0.88],
       [0.28, 0.72],
       [0.  , 1.  ],
       [0.95, 0.05],
       [0.94, 0.06],
       [0.37,

In [39]:
# Calculate the maximum predicted probability for each compound and compare it to the desired threshold
da = np.amax(prediction_probability, axis=1) > threshold

In [40]:
da

array([ True,  True, False, False, False,  True,  True, False, False,
        True, False, False,  True, False, False,  True, False, False,
       False,  True,  True, False, False, False, False, False,  True,
        True,  True, False,  True, False, False, False, False,  True,
       False, False,  True, False, False,  True,  True, False,  True,
        True,  True, False, False, False, False, False, False,  True,
        True, False,  True, False,  True,  True, False, False,  True,
        True, False])

#### Time to calculate some statistics for the prediciton probabilities

In [60]:
print("Accuracy = ", accuracy_score(np.asarray(y_testing)[da], prediction_random_forest[da]))
print("MCC = ", matthews_corrcoef(np.asarray(y_testing)[da], prediction_random_forest[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(y_testing)[da], prediction_random_forest[da]))
print("Coverage = ", sum(da) / len(da))

Accuracy =  0.813953488372093
MCC =  0.6600586246040802
Kappa =  0.6348195329087049
Coverage =  0.6615384615384615


## Second Model: Support-Vector Machines (SVM)

In [45]:
parameter_grid = [{'kernel': ['rbf'], 'gamma': [10 ** i for i in range(-6, 0)],
                     'C': [10 ** i for i in range(0, 5)]},
                    {'kernel': ['linear'], 'C': [10 ** i for i in range(0, 5)]}]
svm = GridSearchCV(SVC(probability=True), parameter_grid, n_jobs=2, cv=cross_validations, verbose=1)

In [47]:
# run model
svm.fit(x_training, y_training)

Fitting 5 folds for each of 35 candidates, totalling 175 fits
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   14.9s
[Parallel(n_jobs=2)]: Done 175 out of 175 | elapsed:   56.6s finished
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=73, shuffle=False),
             error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=2,
             param_grid=[{'C': [1, 10, 100, 1000, 10000],
                          'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000, 10000],
                          'kernel': ['linear']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [48]:
svm.best_score_

0.7734375

In [49]:
svm.best_params_

{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}

In [53]:
prediction_svm = svm.predict(x_testing)
print("Accuracy = ", accuracy_score(y_testing, prediction_svm))
print("MCC = ", matthews_corrcoef(y_testing, prediction_svm))
print("Kappa = ", cohen_kappa_score(y_testing, prediction_svm))

In [56]:
prediciton_probability = svm.predict_proba(x_testing)
da = np.amax(prediciton_probability, axis=1) > threshold
print("Accuracy = ", accuracy_score(np.asarray(y_testing)[da], prediction_svm[da]))
print("MCC = ", matthews_corrcoef(np.asarray(y_testing)[da], prediction_svm[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(y_testing)[da], prediction_svm[da]))
print("Coverage = ", sum(da) / len(da))

### Third model: Gradient Boosting Classifier (GBM)
#### This will compute the consensus predictions from random forest and SVM models

In [64]:
# setup model
parameter_grid = {"n_estimators": [100, 200, 300, 400, 500]}
gbm = GridSearchCV(GradientBoostingClassifier(subsample=0.5, max_features=0.5), 
                   parameter_grid, n_jobs=2, cv=cross_validations, verbose=1)

In [66]:
# run model
gbm.fit(x_training, y_training)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  25 out of  25 | elapsed:   18.1s finished
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=73, shuffle=False),
             error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=0.5,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_c

In [67]:
gbm.best_score_

0.796875

In [68]:
gbm.best_params_

{'n_estimators': 100}

In [69]:
prediction_gbm = gbm.predict(x_testing)
print("Accuracy = ", accuracy_score(y_testing, prediction_gbm))
print("MCC = ", matthews_corrcoef(y_testing, prediction_gbm))
print("Kappa = ", cohen_kappa_score(y_testing, prediction_gbm))

#### consensus model

In [73]:
prediction_consensus = 1 * (((prediction_random_forest + prediction_svm + prediction_gbm) / 3) >= 0.5)
print("Accuracy = ", accuracy_score(y_testing, prediction_consensus))
print("MCC = ", matthews_corrcoef(y_testing, prediction_consensus))
print("Kappa = ", cohen_kappa_score(y_testing, prediction_consensus))

### Add to the fingerprints some other descriptors and look at the consensus model performance

In [102]:
# calculate some descriptors
descriptors = []
for molecule in molecules:
    descriptors.append([Descriptors.MolLogP(molecule),
                  Descriptors.TPSA(molecule),
                  Descriptors.NHOHCount(molecule),
                  Descriptors.NOCount(molecule),
                  Descriptors.NumHAcceptors(molecule),
                  Descriptors.NumHDonors(molecule),
                  Descriptors.NumRotatableBonds(molecule),
                  Descriptors.NumHeteroatoms(molecule),
                  Descriptors.FractionCSP3(molecule)])
descriptors = np.asarray(descriptors)

In [104]:
x = np.concatenate((x, descriptors), axis=1)
x_training, x_testing, y_training, y_testing = train_test_split(x, y, test_size=0.20, random_state=seed)
scaler = StandardScaler().fit(x_training)
x_training = scaler.transform(x_training)
parameter_grid = {"max_features": [x_training.shape[1] // 10, x_training.shape[1] // 7, x_training.shape[1] // 5, x_training.shape[1] // 3], 
              "n_estimators": [100, 200, 300, 400, 500]}
model = GridSearchCV(RandomForestClassifier(), parameter_grid, n_jobs=2, cv=cross_validations, verbose=1)
model.fit(x_training, y_training)

In [111]:
model.best_score_

0.86328125

In [112]:
x_testing = scaler.transform(x_testing)
prediction = model.predict(x_testing)
print("Accuracy = ", accuracy_score(y_testing, prediction))
print("MCC = ", matthews_corrcoef(y_testing, prediction))
print("Kappa = ", cohen_kappa_score(y_testing, prediction))

In [115]:
# estimate applicability domain and calc stat
prediction_probability = model.predict_proba(x_testing)
da = np.amax(prediction_probability, axis=1) > threshold
print("Accuracy = ", accuracy_score(np.asarray(y_testing)[da], prediction[da]))
print("MCC = ", matthews_corrcoef(np.asarray(y_testing)[da], prediction[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(y_testing)[da], prediction[da]))
print("Coverage = ", sum(da) / len(da))

#### Let's try to analyse which variables are the most important in the model

In [118]:
# rebuild the Random Forest model manually using best parameters to be able to extract additional information from the model
random_forest = RandomForestClassifier(n_estimators=model.best_params_["n_estimators"], 
                           max_features=model.best_params_["max_features"],
                           random_state=seed)
random_forest.fit(x_training, y_training)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=295, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=73, verbose=0,
                       warm_start=False)

In [119]:
importance = random_forest.feature_importances_

In [120]:
importance

array([0.        , 0.00149662, 0.00025045, ..., 0.00751906, 0.01958647,
       0.02424284])

In [121]:
indices = np.argsort(importance)[::-1]

print("Feature ranking:")

# print top 25 features
for i in range(25):
    print("%d.\tfeature_index = %d\t(%f)" % (i + 1, indices[i], importance[indices[i]]))

Feature ranking:
1.	feature_index = 2049	(0.094693)
2.	feature_index = 2058	(0.082216)
3.	feature_index = 2060	(0.043471)
4.	feature_index = 2057	(0.042505)
5.	feature_index = 2051	(0.037312)
6.	feature_index = 2048	(0.031235)
7.	feature_index = 2065	(0.024243)
8.	feature_index = 2055	(0.023781)
9.	feature_index = 2056	(0.022325)
10.	feature_index = 2052	(0.020145)
11.	feature_index = 2061	(0.020015)
12.	feature_index = 2064	(0.019586)
13.	feature_index = 650	(0.018186)
14.	feature_index = 2050	(0.012931)
15.	feature_index = 2059	(0.010798)
16.	feature_index = 2053	(0.008813)
17.	feature_index = 2062	(0.008812)
18.	feature_index = 2054	(0.008456)
19.	feature_index = 2063	(0.007519)
20.	feature_index = 794	(0.007504)
21.	feature_index = 378	(0.006732)
22.	feature_index = 892	(0.005309)
23.	feature_index = 327	(0.005174)
24.	feature_index = 795	(0.005014)
25.	feature_index = 807	(0.004759)


#### Note: Features with numbers 1-2048 are different auto-generated fingerprints
### In the order we defined them, the following feature have the associated indicies

2049 - MolLogP --------------- Rank: 1  
2050 - TPSA(m) --------------- Rank: 4  
2051 - NHOHCount ------------- Rank: 5  
2052 - NOCount --------------- Rank: 10  
2053 - NumHAcceptors --------- Rank: 16  
2054 - NumHDonors ------------ Rank: 18  
2055 - NumRotatableBonds ----- Rank: 8  
2056 - NumHeteroatoms -------- Rank: 9  
2057 - FractionCSP3 ---------- Rank: 4  
